# 差分プライバシーを説明する

機械学習プロジェクトでは、通常、データの洞察を得て、どの変数が予測モデルの構築に役立つ可能性が最も高いかを判断するために、データ分析の反復プロセスが含まれます。データの分析には、通常、変数の統計的分布とそれらの間の関係についての洞察を提供する集計関数と統計関数が含まれます。大量のデータでは、集計によって抽象化のレベルが提供されます。しかし、データ量が少ない場合や分析を繰り返す場合には、集計した結果でも個々の観測値の詳細が明らかになることがあります。

*差分プライバシー*は、データに「ノイズ」を加えることによって、個々のデータ ポイントのプライバシーを保護するように設計された手法です。目標は、データの全体的な統計的構成の一貫性を維持しながら、個々の値にプライバシーを提供するために十分なノイズを確実に追加し、集計が元の生データで使用した場合と統計的に類似した結果を生成することです。

## SmartNoise SDK をインストールする

[*SmartNoise*](https://smartnoise.org/) は OpenDP のツールキットです。Microsoft、ハーバード大学などの研究者が共同で進めているこのプロジェクトは、データ分析や機械学習プロジェクトで差分プライバシーを利用するための基礎を提供することを目的としています。

> **注**: SmartNoise は現時点では開発の初期段階にあります。

まず、SmartNoise Python SDK パッケージをインストールします。

In [ ]:
!pip install opendp-smartnoise==0.1.2

## データを読み込む

次に、生データを見てみましょう。この例では、糖尿病の検査を受けた患者の 1 万件の記録があります。

In [ ]:
import pandas as pd

data_path = 'data/diabetes.csv'
diabetes = pd.read_csv(data_path)
diabetes.describe()

上記のコードからの出力は、糖尿病データセットにおける変数の主要な概要の統計情報を示します。

## 分析を実行する

SmartNoiseを使用すると、ソース データにノイズを追加する分析を作成できます。ノイズがどのように追加されるかという基礎となる数学は非常に複雑ですが、SmartNoiseはその詳細のほとんどを処理してくれます。ただし、知っておくと便利な概念がいくつかあります。

- **上限と下限**: *クランプ*は、変数の値の上限と下限を設定するために使用されます。これは、SmartNoise によって生成されるノイズが元のデータの予想される分布と一致することを保証するために必要です。
- **サンプル サイズ**: 一部の集計で一貫性のある差分プライベート データを生成するには、SmartNoise は生成されるデータ・サンプルのサイズを知る必要があります。
- **イプシロン**: 簡単に説明すると、*イプシロン*は負ではない値で、データに追加されるノイズの量を逆算します。イプシロンが低いほどプライバシー レベルの高いデータセットとなり、イプシロンが高いほど元のデータに近いデータセットとなります。一般に、0 から 1 の間のイプシロン値を使用する必要があります。イプシロンは、*デルタ*と呼ばれる別の値と相関しており、この値は分析によって生成されたレポートが完全に非公開ではない可能性を示します。

これらの概念を念頭に置いて、次のコードを調べて実行します。これにより、分析が作成され、差分プライベート データから平均 **Age** 値がレポートされます。元の生データの実際の平均値も比較のために表示されます。

In [ ]:
import opendp.smartnoise.core as sn

cols = list(diabetes.columns)
age_range = [0.0, 120.0]
samples = len(diabetes)

with sn.Analysis() as analysis:
    # データを読み込む
    data = sn.Dataset(path=data_path, column_names=cols)
    
    # Convert Age to float
    age_dt = sn.to_float(data['Age'])
    
    # get mean of age
    age_mean = sn.dp_mean(data = age_dt,
                          privacy_usage = {'epsilon': .50},
                          data_lower = age_range[0],
                          data_upper = age_range[1],
                          data_rows = samples
                         )
    
analysis.release()

# 平均年齢の差分プライベート推定を印刷する
print("Private mean age:",age_mean.value)

# 実際の平均年齢を印刷する
print("Actual mean age:",diabetes.Age.mean())

## ヒストグラムを使用してデータ分布を調べる

データを分析する際は、ヒストグラムを使って変数の分布を調べるのが一般的です。

たとえば、糖尿病データセットにおける年齢の真の分布を見てみましょう。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

ages = list(range(0, 130, 10))
age = diabetes.Age

# 10 年ビンでヒストグラムをプロットする
n_age, bins, patches = plt.hist(age, bins=ages, color='blue', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('True Age Distribution')
plt.show()
print(n_age.astype(int))

次に、年齢の差分プライベート ヒストグラムを比較してみましょう。

In [ ]:
import matplotlib.pyplot as plt

with sn.Analysis() as analysis:
    data = sn.Dataset(path = data_path, column_names = cols)

    age_histogram = sn.dp_histogram(
            sn.to_int(data['Age'], lower=0, upper=120),
            edges = ages,
            upper = 10000,
            null_value = -1,
            privacy_usage = {'epsilon': 0.5}
        )
    
analysis.release()

plt.ylim([0,7000])
width=4
agecat_left = [x + width for x in ages]
agecat_right = [x + 2*width for x in ages]
plt.bar(list(range(0,120,10)), n_age, width=width, color='blue', alpha=0.7, label='True')
plt.bar(agecat_left, age_histogram.value, width=width, color='orange', alpha=0.7, label='Private')
plt.legend()
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

print(age_histogram.value)

ヒストグラムは、差分プライベート データに基づくレポートが、生データからのレポートと同じ洞察を提供することを保証するために十分に類似しています。

## 共分散を計算する

分析のもう 1 つの一般的な目標は、変数間の関係を確立することです。SmartNoise は、これに役立つ差分プライベート*共分散*関数を提供します。

In [ ]:
with sn.Analysis() as analysis:
    sn_data = sn.Dataset(path = data_path, column_names = cols)

    age_bp_cov_scalar = sn.dp_covariance(
                left = sn.to_float(sn_data['Age']),
                right = sn.to_float(sn_data['DiastolicBloodPressure']),
                privacy_usage = {'epsilon': 1.0},
                left_lower = 0.,
                left_upper = 120.,
                left_rows = 10000,
                right_lower = 0.,
                right_upper = 150.,
                right_rows = 10000)
analysis.release()
print('Differentially private covariance: {0}'.format(age_bp_cov_scalar.value[0][0]))
print('Actual covariance', diabetes.Age.cov(diabetes.DiastolicBloodPressure))

この例では、**Age** と **DisatolicBloodPressure** の間の共分散は正であり、高齢患者の方が血圧が高い傾向があることを示しています。

## SQL クエリを使用する

**分析**機能に加えて、SmartNoise では、データ ソースに対して SQL クエリを使用して、差分プライベート集計結果を取得できます。

最初に、データ スキーマ内のテーブルのメタデータを定義する必要があります。これは、**/metadata** フォルダーの **diabetes.yml** ファイルなどの .yml ファイルで実行できます。メタデータは、データ型、数値フィールドの最小値と最大値など、テーブル内のフィールドを記述します。

In [ ]:
from opendp.smartnoise.metadata import CollectionMetadata

meta = CollectionMetadata.from_file('metadata/diabetes.yml')
print (meta)

メタデータを定義すると、クエリできる*リーダー*を作成できます。次の例では、Pandas データフレームから生データを読み取る **PandasReader** と、**PandasReader** に差分プライバシー層を追加する **PrivateReader** を作成します。

In [ ]:
from opendp.smartnoise.sql import PandasReader, PrivateReader

reader = PandasReader(diabetes, meta)
private_reader = PrivateReader(reader, meta)
print('Readers ready.')

これで、集計された結果セットを返す SQL クエリをプライベート リーダーに送信できるようになりました。

In [ ]:
query = 'SELECT Diabetic, AVG(Age) AS AvgAge FROM diabetes.diabetes GROUP BY Diabetic'

result_dp = private_reader.execute(query)
print(result_dp)

結果を生データからの同じ集計と比較してみましょう。

In [ ]:
result = reader.execute(query)
print(result)

明示的なイプシロン値を指定することで、**PrivateReader** の動作をカスタマイズできます。

イプシロンの値が高い (プライバシーが低い) リーダーと、イプシロンの値が低い (プライバシーが高い) リーダーを試してみましょう。

In [ ]:
low_privacy_reader = PrivateReader(reader, meta, 5.0)  # large epsilon, less privacy
result = low_privacy_reader.execute(query)
print(result)
print()

high_privacy_reader = PrivateReader(reader, meta, 0.1)  # smaller epsilon, more privacy
result = high_privacy_reader.execute(query)
print(result)

イプシロンが高い (プライバシーが低い) リーダーの結果はイプシロンが低い (プライバシーが高い) リーダーの結果よりも生データからの真の結果に近いことに注意してください。

## 詳細情報

SmartNoise を使用した差分プライバシーの詳細については、[https://smartnoise.org](https://smartnoise.org/) を参照してください